In [8]:
import logging
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus
logging.basicConfig()

In [9]:
texts = []
f = open('../dataset/restaurant.txt')
for line in f:
    texts.append(line.replace("\n", ""))

In [10]:
# Preprocess data
max_length = 10000   # Limit of 10k words per document
tokens, vocab = preprocess.tokenize(texts, max_length, merge=False,
                                   n_threads=4)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [11]:
# print(vocab)

In [12]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=30)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)


In [13]:
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 300
fn_wordvc = '../../GoogleNews-vectors-negative300.bin'
vectors, s, f = corpus.compact_word_vectors(vocab, filename=fn_wordvc)

err


In [14]:
# Save all of the preprocessed files
pickle.dump(vocab, open('res/vocab.pkl', 'wb'))
pickle.dump(corpus, open('res/corpus.pkl', 'wb'))
np.save("res/flattened", flattened)
np.save("res/doc_ids", doc_ids)
np.save("res/pruned", pruned)
np.save("res/bow", bow)
np.save("res/vectors", vectors)